# Let's phase it
We will look into nmrglue auto phasing algorithms and also we'll see how to build one from scratch.
Also, why not trying building an interactive manual phasing algorithm, it's not that hard.

### Import libraries

In [32]:
# import libraries
import os
import os.path as op

import numpy as np
from bokeh.io import output_notebook, reset_output, show
from bokeh.plotting import figure

# import from our own local module
from procnmr.plotting import bkplot
import nmrglue as ng

In [33]:
reset_output()
output_notebook()

Loading BokehJS ...

In [34]:
datadir = 'data/lurasidona'

# load fid data
par, fid = ng.bruker.read(op.join(datadir, '1'))

# remove digital filter
fid = ng.bruker.remove_digital_filter(par, fid)

# fft
ft = ng.proc_base.fft(fid)

In [35]:
fig_fid = bkplot(x=range(fid.shape[-1]), y=fid.real)
show(fig_fid)

In [36]:
fig_ft = bkplot(x=range(ft.shape[-1]), y=ft.real)
show(fig_ft)

Let's try auto phasing algorithm from `nmrglue`

In [43]:
ft2, opt_phase = ng.proc_autophase.autops(ft, 'acme', p0=0, p1=0, return_phases=True)

Optimization terminated successfully.
         Current function value: 6221.509564
         Iterations: 115
         Function evaluations: 220


In [46]:
print('Optimal Phase values from nmrglue: %f, %f' % (opt_phase[0], opt_phase[1]))

Optimal Phase values from nmrglue: -220.220845, 256.476467


In [47]:
fig_ft2 = bkplot(x=range(ft2.shape[-1]), y=ft2.real)
show(fig_ft2)

In [48]:
import scipy.optimize as opt


def phase_corr(data, p0, p1) :
    p0 = p0 * np.pi / 180. 
    p1 = p1 * np.pi / 180.
    size = data.shape[-1]
    corr = np.exp(-1.0j * (p0 + (p1 * np.linspace(0, 1, size)))).astype(data.dtype)
    return corr * data

def dx(data):
    z = np.empty_like(data)
    z[..., 0] = data[..., 1] - data[..., 0]    # first point
    z[..., -1] = data[..., -1] - data[..., -2]  # last point
    z[..., 1:-1] = data[..., 2:] - data[..., 1:-1]  # interior
    return z

def entropy(data) :
    d1 = dx(data.real)
    norm = sum(np.abs(d1))
    h = np.abs(d1) / norm
    #plotspec(h)
    return -sum(h * np.log(h))
    
def penalty(data) :
    return sum([i**2 for i in data.real if i < 0])
    
def E(data, alpha, beta, gamma, ph0, ph1) :
    f = phase_corr(data, p0=ph0, p1=ph1)
    e = alpha * entropy(f)
    i = beta * tot_abs_int(f)
    p = gamma * penalty(f)

    return e + i + p
    
def minE(x, *args) :
    return E(args[0], args[1], args[2], args[3], x[0], x[1])

def tot_abs_int(data) :
    ''' calculates the total absolute intensity of the spectrum '''
    return np.abs(data.real).sum()

In [49]:
def auto_phase(data, ini_phase=[0,0], method='CG'):
    alpha = 1
    beta = 1
    gamma = 1
    args = (data, alpha, beta, gamma)
    opt_phase = opt.minimize(minE, x0=ini_phase, args=args, method=method)
    
    return opt_phase

In [50]:
res = auto_phase(ft)
res

     fun: 3490922059267.841
     jac: array([ -65536., -262144.])
 message: 'Desired error not necessarily achieved due to precision loss.'
    nfev: 242
     nit: 15
    njev: 76
  status: 2
 success: False
       x: array([ 220.21110695, -256.45204826])

In [51]:
res.x

array([ 220.21110695, -256.45204826])

In [52]:
ft2 = phase_corr(ft, res.x[0], res.x[1])
fig_ft2 = bkplot(x=range(ft2.shape[-1]), y=ft2.real)
show(fig_ft2)

Even though previous chapters method and this one gives very similar results in how the spectra looks, the zero and first order phase values are different. 

### Manual phasing


In [79]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider

x = range(ft2.shape[-1]) 
re = ft2.real
im = ft2.imag

source = ColumnDataSource(data=dict(x=x, y=re))

plot = figure(plot_width=800, plot_height=500)
plot.line('x', 'y', source=source, color='firebrick', line_width=1, line_alpha=1.0)

ph0_slider = Slider(start=0, end=360, value=1, step=.01, title="Ph0")
ph1_slider = Slider(start=0, end=360, value=1, step=.01, title="Ph1")

# phase correction cosine form
# re = cos*re + sin*im
# im = -sin*re + cos*im

update_curve = CustomJS(args=dict(source=source, re=re, im=im, ph0_slider=ph0_slider, ph1_slider=ph1_slider), code="""
    var data = source.data;
    var re = re;
    var im = im;
    var phi = 0.0;
    var ph0 = ph0_slider.value/180*Math.PI;
    var ph1 = ph1_slider.value/180*Math.PI;
    var x = data['x']
    var y = data['y']
    for (var i = 0; i < y.length; i++) {
        phi = ph0 + ph1 * i / y.length
        y[i] = Math.cos(phi)*re[i] + Math.sin(phi)*im[i]
    }
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")

ph0_slider.js_on_change('value', update_curve)
ph1_slider.js_on_change('value', update_curve)


show(column(ph0_slider, ph1_slider, plot))

This is a nice starting point, but it has a few problems. Can you spot them? 
In what ways could we improve it?

go [next](05_baseline_correction.ipynb) or go [home](00_introduction.ipynb), there's no going [back](03_basic_processing.ipynb)